In [ ]:

import numpy as np

%load_ext autoreload

%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from skimage.exposure import rescale_intensity
import dask
from sentinelhub import CRS, BBox, DataCollection, SHConfig
from sentinelhub import SentinelHubCatalog
from sentinelhub import SHConfig
from oxeo.core.stac import landsat
from oxeo.core.stac.constants import USWEST_URL ,ELEMENT84_URL
uswest_config = SHConfig()
uswest_config.sh_base_url = USWEST_URL

uswest_catalog = SentinelHubCatalog(config=uswest_config)
eu_catalog = SentinelHubCatalog(SHConfig())
from oxeo.core.data import get_aoi_from_landsat_shub_catalog, get_aoi_from_stac_catalog
import os
os.environ["AWS_REQUEST_PAYER"] = "requester"

from oxeo.core.utils import get_bounding_box

from oxeo.water.models.segmentation import Segmentation2DPredictor, DaskSegmentationPredictor, reconstruct_image_from_patches
from oxeo.core.models.tile import load_tile_from_stac_as_dict, load_aoi_from_stac_as_dict, tile_from_id, TilePath, tile_to_geom
from oxeo.core import data
import matplotlib.pyplot as plt

box = get_bounding_box({"geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              14.911966323852539,
              37.30573714593416
            ],
            [
              14.987583160400392,
              37.30573714593416
            ],
            [
              14.987583160400392,
              37.345050859282736
            ],
            [
              14.911966323852539,
              37.345050859282736
            ],
            [
              14.911966323852539,
              37.30573714593416
            ]
          ]
        ]
      }})
      
bbox = BBox(box, crs=CRS.WGS84)#BBox([49.9604, 44.7176, 51.0481, 45.2324], crs=CRS.WGS84) #BBox((-71.40254974365233, -46.9537775782648, -71.26213073730467, -46.89163931213445), crs=CRS.WGS84)#
time_interval = "2020-12-10", "2021-02-01" #"1994-12-10", "1995-02-01"
search_params = {
}


In [ ]:
import toolz

In [ ]:
from distributed import Client

client = Client(n_workers=4, threads_per_worker=1, memory_limit='16GB')
client

In [ ]:
s2_aoi = get_aoi_from_stac_catalog(catalog=ELEMENT84_URL,
                                   data_collection="sentinel-s2-l2a-cogs",
                                   bbox=bbox,
                                   time_interval=time_interval,
                                   search_params={})


In [ ]:
s2_aoi

In [ ]:
def pad_xarray_to(arr, pad_to):
    pad_top = abs(arr.shape[-2] % -pad_to)
    pad_right = abs(arr.shape[-1] % -pad_to)
    
    return arr.pad(x=(0, pad_right), y=(0, pad_top))
    
def create_patches_from_xarray(arr, patch_size):
        """
        Creates patches from a matrix.
        :param m: matrix
        :param patch_size: patch size
        :return: list of patches
        """
        patches = []
        for t in range(0, arr.shape[0]):
            for i in range(0, arr.shape[-2], patch_size):
                for j in range(0, arr.shape[-1], patch_size):
                    patches.append(arr[t, :, i : i + patch_size, j : j + patch_size])
        return patches

In [ ]:
from oxeo.core.constants import BAND_PREDICTOR_ORDER
s2_predictor = DaskSegmentationPredictor(ckpt_path="../data/semseg_epoch_012.ckpt", fs=None,
                                    bands=BAND_PREDICTOR_ORDER["sentinel-2"])

In [ ]:
preds, aoi = s2_predictor.predict_stac_aoi(constellation="sentinel-2", catalog=ELEMENT84_URL,
                                   data_collection="sentinel-s2-l2a-cogs",
                                   bbox=bbox,
                                   time_interval=time_interval,
                                   search_params={})

In [ ]:
aoi

In [ ]:
res = client.compute(preds)

In [ ]:
stack = np.vstack([e.result() for e in res])
mask = reconstruct_image_from_patches(stack, aoi.shape[0], aoi.shape[-2], aoi.shape[-1], patch_size=250 )

In [ ]:
img = aoi[1,[3,2,1],:,:].values.transpose(1,2,0)
plt.imshow(rescale_intensity(img,(np.nanmin(img), np.nanmax(img))))

In [ ]:
plt.imshow(mask[1,:,:])